# MusicAPI
---
## Starter Code to Generate a List of Performances
### NOTE:
change jupyter notebook settings with code line below:

jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10

^^ this increases data rate limit to handle JSON, if your setup can handle it^^

In [75]:
# Dependencies
import requests
import json
import time
import pandas as pd
import numpy as np
# Possibly need to...Import citipy to determine the cities based on latitude and longitude from venues
from citipy import citipy
import csv

# Import ticketmaster API key
from api_keys import ticketmaster_key

url = "https://app.ticketmaster.com/discovery/v2/events.json?"
#or "https://app.ticketmaster.com/discovery/v2/events?"

##example https://app.ticketmaster.com/discovery/v2/events.json?classificationName=music&dmaId=324&apikey={apikey}

##sampleQueryURL https://app.ticketmaster.com/discovery/v2/events?apikey=___apikey____
#^^do we need to include "event.json?" or only "events?"^^
##&locale=*
#^^do we need "&locale=*"^^
##&startDateTime=2023-12-11T15:54:00Z
##&endDateTime=2023-12-15T15:54:00Z
##&city=Washington
##&countryCode=US
##&stateCode=DC&classificationName=music
##&sort=date,asc

# Store type of event
#event_type = "music", can just be included in URL build...

# Store location/city
dc_city = "Washington"
dc_state_code = "DC"

# Query of music events between a begin and end date, for now next week...then ideally for one month
begin_dt = "2023-12-01T02:00:00Z"
#^^2AM military time because some shows run over to the nextday...^^
end_dt = "2023-12-31T23:59:00Z"

# Build URL
query_url = f"{url}apikey={ticketmaster_key}&classificationName=music&startDateTime={begin_dt}&endDateTime={end_dt}&size=200&city={dc_city}&countryCode=US&stateCode={dc_state_code}&sort=date,asc"

# Requirement 1: Get date, time, venue location, venue coordinates, and image for music events within timeframe

### Cities
- New York
- Dc
- Atlanta
- Los Angeles
- Nashville
- Austin

### cool idea to add is weather for the week to plan accordingly (transport[walk, vehicle], outfit)

In [50]:
nyc_query = f"{url}apikey={ticketmaster_key}&classificationName=music&startDateTime={begin_dt}&endDateTime={end_dt}&size=200&dmaId=345&sort=date,asc"
nyc_shows = requests.get(nyc_query).json()
nyc_list = []
#nyc_shows["name"][, images[0], dates["start"
#what we need...
for show in nyc_shows['_embedded']['events']:
    nyc_name = show["name"]
    #image = nyc_shows["image"[0]] 
    #^^can we save/retrieve image? it would be a nice touch...^^
    nyc_show_date = show['dates']['start']['localDate'] #done
    try:
        #show['genre']
        nyc_show_time = show['dates']['start']['localTime'] #done
    except KeyError:
        print('Undefined')
    #^^we need to see what we can do to stil include music performances even though no time is announced...^^
    nyc_show_genre = show['classifications'][0]['genre']['name'] #done
    ##show_subgenre = show['classifications'][0]['subGenre']['name'] #done
    #^^how do we make an exception or skip if the subgenre is not applicable?^^
    nyc_show_venue = show['_embedded']['venues'][0]['name'] #done
    nyc_venue_city = show['_embedded']['venues'][0]['city']['name'] #done
    nyc_venue_state = show['_embedded']['venues'][0]['state']['stateCode'] #done
    #^^in case we need these(city, state)..^^
    nyc_venue_long = show['_embedded']['venues'][0]['location']['longitude'] #done
    nyc_venue_lat = show['_embedded']['venues'][0]['location']['latitude'] #done
    nyc_performances = {
        'Performing': nyc_name,
        'date': nyc_show_date,
        'time': nyc_show_time,
        'genre': nyc_show_genre,
        #'subgenre': show_subgenre,
        'venue': nyc_show_venue,
        'city': nyc_venue_city,
        'state': nyc_venue_state,
        'long': nyc_venue_long,
        'lat': nyc_venue_lat,
    }
    nyc_list.append(nyc_performances)
print(nyc_list)
#nyc_list
#nyc_shows

Undefined
Undefined
[{'Performing': 'girlhouse', 'date': '2023-12-06', 'time': '18:00:00', 'genre': 'Rock', 'venue': 'Mercury Lounge', 'city': 'New York', 'state': 'NY', 'long': '-73.986806', 'lat': '40.722036'}, {'Performing': 'The Maine', 'date': '2023-12-06', 'time': '18:30:00', 'genre': 'Rock', 'venue': 'Irving Plaza Powered By Verizon 5G', 'city': 'New York', 'state': 'NY', 'long': '-73.9884469', 'lat': '40.734917'}, {'Performing': 'Energy Curfew Music Hour W/ Chris Thile And Punch Brothers', 'date': '2023-12-06', 'time': '19:00:00', 'genre': 'Variety', 'venue': 'Minetta Lane Theatre', 'city': 'New York', 'state': 'NY', 'long': '-74.0012676', 'lat': '40.7303577'}, {'Performing': 'GIGGS', 'date': '2023-12-06', 'time': '19:00:00', 'genre': 'Hip-Hop/Rap', 'venue': 'Bowery Ballroom', 'city': 'New York', 'state': 'NY', 'long': '-73.993542', 'lat': '40.720364'}, {'Performing': 'Leanna Firestone', 'date': '2023-12-06', 'time': '19:00:00', 'genre': 'Pop', 'venue': 'Gramercy Theatre', 'cit

In [51]:
len(nyc_list)

200

In [52]:
#page = 0
#or __stuoooff__ in __stuffs__:
   #page += 1
nyc_query_page2 = f"{url}apikey={ticketmaster_key}&classificationName=music&startDateTime={begin_dt}&endDateTime={end_dt}&size=200&page=1&dmaId=345&sort=date,asc"
nyc_shows_page2 = requests.get(nyc_query_page2).json()
for show in nyc_shows_page2['_embedded']['events']:
    nyc_name = show["name"]
    #image = nyc_shows["image"[0]] 
    #^^can we save/retrieve image? it would be a nice touch...^^
    nyc_show_date = show['dates']['start']['localDate'] #done
    try:
        #show['genre']
        nyc_show_time = show['dates']['start']['localTime'] #done
    except KeyError:
        print('Undefined')
    #^^we need to see what we can do to stil include music performances even though no time is announced...^^
    try:
        #show['genre']
        nyc_show_genre = show['classifications'][0]['genre']#['name'] #done
    except KeyError:
        print('Undefined')
        
    ##show_subgenre = show['classifications'][0]['subGenre']['name'] #done
    #^^how do we make an exception or skip if the subgenre is not applicable?^^
    nyc_show_venue = show['_embedded']['venues'][0]['name'] #done
    nyc_venue_city = show['_embedded']['venues'][0]['city']['name'] #done
    nyc_venue_state = show['_embedded']['venues'][0]['state']['stateCode'] #done
    #^^in case we need these(city, state)..^^
    nyc_venue_long = show['_embedded']['venues'][0]['location']['longitude'] #done
    nyc_venue_lat = show['_embedded']['venues'][0]['location']['latitude'] #done
    nyc_performances = {
        'Performing': nyc_name,
        'date': nyc_show_date,
        'time': nyc_show_time,
        'genre': nyc_show_genre,
        #'subgenre': show_subgenre,
        'venue': nyc_show_venue,
        'city': nyc_venue_city,
        'state': nyc_venue_state,
        'long': nyc_venue_long,
        'lat': nyc_venue_lat,
    }
    nyc_list.append(nyc_performances)
print(nyc_list)

Undefined
Undefined
Undefined
Undefined
[{'Performing': 'girlhouse', 'date': '2023-12-06', 'time': '18:00:00', 'genre': 'Rock', 'venue': 'Mercury Lounge', 'city': 'New York', 'state': 'NY', 'long': '-73.986806', 'lat': '40.722036'}, {'Performing': 'The Maine', 'date': '2023-12-06', 'time': '18:30:00', 'genre': 'Rock', 'venue': 'Irving Plaza Powered By Verizon 5G', 'city': 'New York', 'state': 'NY', 'long': '-73.9884469', 'lat': '40.734917'}, {'Performing': 'Energy Curfew Music Hour W/ Chris Thile And Punch Brothers', 'date': '2023-12-06', 'time': '19:00:00', 'genre': 'Variety', 'venue': 'Minetta Lane Theatre', 'city': 'New York', 'state': 'NY', 'long': '-74.0012676', 'lat': '40.7303577'}, {'Performing': 'GIGGS', 'date': '2023-12-06', 'time': '19:00:00', 'genre': 'Hip-Hop/Rap', 'venue': 'Bowery Ballroom', 'city': 'New York', 'state': 'NY', 'long': '-73.993542', 'lat': '40.720364'}, {'Performing': 'Leanna Firestone', 'date': '2023-12-06', 'time': '19:00:00', 'genre': 'Pop', 'venue': 'Gra

In [53]:
#append rest of nyc month
len(nyc_list)

327

In [54]:
# Convert nyc_shows data into a Pandas DataFrame
nyc_data_df = pd.DataFrame(nyc_list)
# confirm Record Count
nyc_data_df.count()

Performing    327
date          327
time          327
genre         327
venue         327
city          327
state         327
long          327
lat           327
dtype: int64

In [55]:
# Display nyc_shows df
nyc_data_df.head()

,Performing,date,time,genre,venue,city,state,long,lat
0,girlhouse,2023-12-06,18:00:00,Rock,Mercury Lounge,New York,NY,-73.986806,40.722036
1,The Maine,2023-12-06,18:30:00,Rock,Irving Plaza Powered By Verizon 5G,New York,NY,-73.9884469,40.734917
2,Energy Curfew Music Hour W/ Chris Thile And Pu...,2023-12-06,19:00:00,Variety,Minetta Lane Theatre,New York,NY,-74.0012676,40.7303577
3,GIGGS,2023-12-06,19:00:00,Hip-Hop/Rap,Bowery Ballroom,New York,NY,-73.993542,40.720364
4,Leanna Firestone,2023-12-06,19:00:00,Pop,Gramercy Theatre,New York,NY,-73.98501292,40.73975368


In [56]:
# Export the dc_shows df into a csv
nyc_data_df.to_csv("../cities_data/nyc_show_data.csv", index_label="Show_ID")

In [57]:
dc_shows = requests.get(query_url).json()
#print(dc_shows['_embedded']['events'][1]['name']) got it
#print(dc_shows['_embedded']['events'][1]['dates']['start'])
dc_list = []
#dc_shows["name"][, images[0], dates["start"
#what we need...
for show in dc_shows['_embedded']['events']:
    name = show["name"]
    #image = dc_shows["image"[0]] 
    #^^can we save/retrieve image? it would be a nice touch...^^
    show_date = show['dates']['start']['localDate'] #done
    show_time = show['dates']['start']['localTime'] #done
    show_genre = show['classifications'][0]['genre']['name'] #done
    ##show_subgenre = show['classifications'][0]['subGenre']['name'] #done
    #^^how do we make an exception or skip if the subgenre is not applicable?^^
    show_venue = show['_embedded']['venues'][0]['name'] #done
    venue_city = show['_embedded']['venues'][0]['city']['name'] #done
    venue_state = show['_embedded']['venues'][0]['state']['stateCode'] #done
    #^^in case we need these(city, state)..^^
    venue_long = show['_embedded']['venues'][0]['location']['longitude'] #done
    venue_lat = show['_embedded']['venues'][0]['location']['latitude'] #done
    performances = {
        'Performing': name,
        'date': show_date,
        'time': show_time,
        'genre': show_genre,
        #'subgenre': show_subgenre,
        'venue': show_venue,
        'city': venue_city,
        'state': venue_state,
        'long': venue_long,
        'lat': venue_lat,
    }
    dc_list.append(performances)
dc_list
#dc_shows

[{'Performing': 'LaMP: Russ Lawton, Scott Metzger, Ray Paczkowski',
  'date': '2023-12-06',
  'time': '18:30:00',
  'genre': 'Rock',
  'venue': 'The Atlantis',
  'city': 'Washington',
  'state': 'DC',
  'long': '-77.02241',
  'lat': '38.91819'},
 {'Performing': 'Au5',
  'date': '2023-12-06',
  'time': '22:00:00',
  'genre': 'Dance/Electronic',
  'venue': 'Soundcheck',
  'city': 'Washington',
  'state': 'DC',
  'long': '-77.032811',
  'lat': '38.902135'},
 {'Performing': 'Neal Francis',
  'date': '2023-12-07',
  'time': '18:30:00',
  'genre': 'Rock',
  'venue': 'The Atlantis',
  'city': 'Washington',
  'state': 'DC',
  'long': '-77.02241',
  'lat': '38.91819'},
 {'Performing': 'Stephen Sanchez',
  'date': '2023-12-07',
  'time': '19:00:00',
  'genre': 'Rock',
  'venue': '9:30 CLUB',
  'city': 'Washington',
  'state': 'DC',
  'long': '-77.02373',
  'lat': '38.91795'},
 {'Performing': 'SCREAM w/ Soulside',
  'date': '2023-12-07',
  'time': '19:30:00',
  'genre': 'Rock',
  'venue': 'Black 

In [58]:
len(dc_list)

100

In [59]:
#dc_shows.json()
#dc_shows
#response
#print(json.dumps(dc_shows, indent = 4))

### Print the count of different genres of city show count
### maybe at the end we can possibly show a pie or bar chart of the different genres playing in city
### from here we can findout what genres are more popular or are booked/planned for city and can tell us more about the type of music the city demographic and surrouding region like to listen.

In [26]:
# Convert dc_shows data into a Pandas DataFrame
dc_data_df = pd.DataFrame(dc_list)
# Show Record Count
dc_data_df.count()

Performing    103
date          103
time          103
genre         103
venue         103
city          103
state         103
long          103
lat           103
dtype: int64

In [27]:
# Display dc_shows df
dc_data_df.head()

,Performing,date,time,genre,venue,city,state,long,lat
0,La Goony Chonga,2023-12-05,18:30:00,Hip-Hop/Rap,The Atlantis,Washington,DC,-77.02241,38.91819
1,Big Freedia's Christmas in Central City Tour,2023-12-05,19:00:00,Hip-Hop/Rap,9:30 CLUB,Washington,DC,-77.02373,38.91795
2,"LaMP: Russ Lawton, Scott Metzger, Ray Paczkowski",2023-12-06,18:30:00,Rock,The Atlantis,Washington,DC,-77.02241,38.91819
3,Cynthia Erivo,2023-12-06,20:00:00,Other,Kennedy Center - Eisenhower Theater,Washington,DC,-77.049004,38.900501
4,Durry w/ Mega Mango,2023-12-06,20:00:00,Other,Union Stage,Washington,DC,-77.019798,38.877998


In [31]:
# Export the dc_shows df into a csv
dc_data_df.to_csv("../cities_data/dc_show_data.csv", index_label="Show_ID")

In [60]:
atl_query = f"{url}apikey={ticketmaster_key}&classificationName=music&startDateTime={begin_dt}&endDateTime={end_dt}&size=200&dmaId=220&sort=date,asc"
atl_shows = requests.get(atl_query).json()
atl_list = []
for show in atl_shows['_embedded']['events']:
    atl_name = show["name"]
    #image = nyc_shows["image"[0]] 
    #^^can we save/retrieve image? it would be a nice touch...^^
    atl_show_date = show['dates']['start']['localDate'] #done
    try:
        #show['genre']
        atl_show_time = show['dates']['start']['localTime'] #done
    except KeyError:
        print('Undefined')
    #^^we need to see what we can do to stil include music performances even though no time is announced...^^
    try:
        #show['genre']
        atl_show_genre = show['classifications'][0]['genre']['name'] #done
    except KeyError:
        print('Undefined')
        
    ##show_subgenre = show['classifications'][0]['subGenre']['name'] #done
    #^^how do we make an exception or skip if the subgenre is not applicable?^^
    atl_show_venue = show['_embedded']['venues'][0]['name'] #done
    atl_venue_city = show['_embedded']['venues'][0]['city']['name'] #done
    atl_venue_state = show['_embedded']['venues'][0]['state']['stateCode'] #done
    #^^in case we need these(city, state)..^^
    atl_venue_long = show['_embedded']['venues'][0]['location']['longitude'] #done
    atl_venue_lat = show['_embedded']['venues'][0]['location']['latitude'] #done
    atl_performances = {
        'Performing': atl_name,
        'date': atl_show_date,
        'time': atl_show_time,
        'genre': atl_show_genre,
        #'subgenre': show_subgenre,
        'venue': atl_show_venue,
        'city': atl_venue_city,
        'state': atl_venue_state,
        'long': atl_venue_long,
        'lat': atl_venue_lat,
    }
    atl_list.append(atl_performances)
print(atl_list)

[{'Performing': 'Sydney Sprague', 'date': '2023-12-06', 'time': '18:30:00', 'genre': 'Rock', 'venue': 'The Masquerade - Purgatory', 'city': 'Atlanta', 'state': 'GA', 'long': '-84.389887', 'lat': '33.75184'}, {'Performing': 'Mike and the Moonpies', 'date': '2023-12-06', 'time': '20:00:00', 'genre': 'Country', 'venue': 'The Loft', 'city': 'Atlanta', 'state': 'GA', 'long': '-84.3877907', 'lat': '33.7918926'}, {'Performing': 'Sarah Brightman: A Christmas Symphony', 'date': '2023-12-06', 'time': '20:00:00', 'genre': 'Pop', 'venue': 'Atlanta Symphony Hall', 'city': 'Atlanta', 'state': 'GA', 'long': '-84.385493', 'lat': '33.790098'}, {'Performing': 'Pardyalone - I Left You In Minnesota Tour', 'date': '2023-12-06', 'time': '20:00:00', 'genre': 'Hip-Hop/Rap', 'venue': 'Vinyl', 'city': 'Atlanta', 'state': 'GA', 'long': '-84.3877907', 'lat': '33.7918926'}, {'Performing': 'Jazmin Bean', 'date': '2023-12-07', 'time': '19:00:00', 'genre': 'Metal', 'venue': 'The Masquerade - Hell', 'city': 'Atlanta',

In [61]:
# Convert atl_shows data into a Pandas DataFrame
atl_data_df = pd.DataFrame(atl_list)
# confirm Record Count
atl_data_df.count()

Performing    102
date          102
time          102
genre         102
venue         102
city          102
state         102
long          102
lat           102
dtype: int64

In [62]:
# Display dc_shows df
atl_data_df.head()

,Performing,date,time,genre,venue,city,state,long,lat
0,Sydney Sprague,2023-12-06,18:30:00,Rock,The Masquerade - Purgatory,Atlanta,GA,-84.389887,33.75184
1,Mike and the Moonpies,2023-12-06,20:00:00,Country,The Loft,Atlanta,GA,-84.3877907,33.7918926
2,Sarah Brightman: A Christmas Symphony,2023-12-06,20:00:00,Pop,Atlanta Symphony Hall,Atlanta,GA,-84.385493,33.790098
3,Pardyalone - I Left You In Minnesota Tour,2023-12-06,20:00:00,Hip-Hop/Rap,Vinyl,Atlanta,GA,-84.3877907,33.7918926
4,Jazmin Bean,2023-12-07,19:00:00,Metal,The Masquerade - Hell,Atlanta,GA,-84.389887,33.75184


In [63]:
# Export the dc_shows df into a csv
atl_data_df.to_csv("../cities_data/atl_show_data.csv", index_label="Show_ID")

In [64]:
la_query = f"{url}apikey={ticketmaster_key}&classificationName=music&startDateTime={begin_dt}&endDateTime={end_dt}&size=200&dmaId=324&sort=date,asc"
la_shows = requests.get(la_query).json()
la_list = []
for show in la_shows['_embedded']['events']:
    la_name = show["name"]
    #image = nyc_shows["image"[0]] 
    #^^can we save/retrieve image? it would be a nice touch...^^
    la_show_date = show['dates']['start']['localDate'] #done
    try:
        #show['genre']
        la_show_time = show['dates']['start']['localTime'] #done
    except KeyError:
        print('Undefined')
    #^^we need to see what we can do to stil include music performances even though no time is announced...^^
    try:
        #show['genre']
        la_show_genre = show['classifications'][0]['genre']['name'] #done
    except KeyError:
        print('Undefined')
        
    la_show_venue = show['_embedded']['venues'][0]['name'] #done
    la_venue_city = show['_embedded']['venues'][0]['city']['name'] #done
    la_venue_state = show['_embedded']['venues'][0]['state']['stateCode'] #done
    #^^in case we need these(city, state)..^^
    
    try:
        la_venue_long = show['_embedded']['venues'][0]['location']['longitude'] #done
    except KeyError:
        print('Undefined')
    try:
        la_venue_lat = show['_embedded']['venues'][0]['location']['latitude'] #done
    except KeyError:
        print('Undefined')
    
    la_performances = {
        'Performing': la_name,
        'date': la_show_date,
        'time': la_show_time,
        'genre': la_show_genre,
        #'subgenre': show_subgenre,
        'venue': la_show_venue,
        'city': la_venue_city,
        'state': la_venue_state,
        'long': la_venue_long,
        'lat': la_venue_lat,
    }
    la_list.append(la_performances)
#print(nyc_show_genre)
    #^^need nested for or if loop to be able to fill in a "unavailable" value if genre is not inlcuded with performance listing.^^
print(la_list)

Undefined
Undefined
Undefined
Undefined
Undefined
[{'Performing': 'Wendlo', 'date': '2023-12-06', 'time': '18:30:00', 'genre': 'Rock', 'venue': 'The Moroccan Lounge', 'city': 'Los Angeles', 'state': 'CA', 'long': '-118.23365', 'lat': '34.04898'}, {'Performing': "The Gang's All Here Tour With Skid Row And Buckcherry", 'date': '2023-12-06', 'time': '18:30:00', 'genre': 'Metal', 'venue': 'House of Blues Anaheim', 'city': 'Anaheim', 'state': 'CA', 'long': '-117.9229975', 'lat': '33.8095868'}, {'Performing': 'Your Neighbors with special guest 7KY', 'date': '2023-12-06', 'time': '19:00:00', 'genre': 'Rock', 'venue': 'The Peppermint Club', 'city': 'Los Angeles', 'state': 'CA', 'long': '-118.381072', 'lat': '34.076851'}, {'Performing': 'Amber Wild with Hardcastle, and Levi Evans', 'date': '2023-12-06', 'time': '19:00:00', 'genre': 'Rock', 'venue': 'The Echo', 'city': 'Los Angeles', 'state': 'CA', 'long': '-118.260108', 'lat': '34.077729'}, {'Performing': "Jenny Lewis - The Joy'All Ball Tour", 

In [65]:
# confirm first page extraction
len(la_list)

200

In [66]:
la_query_page2 = f"{url}apikey={ticketmaster_key}&classificationName=music&startDateTime={begin_dt}&endDateTime={end_dt}&size=200&page=1&dmaId=324&sort=date,asc"
la_shows_page2 = requests.get(la_query_page2).json()
for show in la_shows_page2['_embedded']['events']:
    la_name = show["name"]
    #image = nyc_shows["image"[0]] 
    #^^can we save/retrieve image? it would be a nice touch...^^
    la_show_date = show['dates']['start']['localDate'] #done
    try:
        #show['genre']
        la_show_time = show['dates']['start']['localTime'] #done
    except KeyError:
        print('Undefined')
    #^^we need to see what we can do to stil include music performances even though no time is announced...^^
    try:
        #show['genre']
        la_show_genre = show['classifications'][0]['genre']#['name'] #done
    except KeyError:
        print('Undefined')
        
    la_show_venue = show['_embedded']['venues'][0]['name'] #done
    la_venue_city = show['_embedded']['venues'][0]['city']['name'] #done
    la_venue_state = show['_embedded']['venues'][0]['state']['stateCode'] #done
    #^^in case we need these(city, state)..^^
    
    try:
        la_venue_long = show['_embedded']['venues'][0]['location']['longitude'] #done
    except KeyError:
        print('Undefined')
    try:
        la_venue_lat = show['_embedded']['venues'][0]['location']['latitude'] #done
    except KeyError:
        print('Undefined')
    
    la_performances = {
        'Performing': la_name,
        'date': la_show_date,
        'time': la_show_time,
        'genre': la_show_genre,
        #'subgenre': show_subgenre,
        'venue': la_show_venue,
        'city': la_venue_city,
        'state': la_venue_state,
        'long': la_venue_long,
        'lat': la_venue_lat,
    }
    la_list.append(la_performances)
#print(nyc_show_genre)
    #^^need nested for or if loop to be able to fill in a "unavailable" value if genre is not inlcuded with performance listing.^^
print(la_list)

Undefined
Undefined
[{'Performing': 'Wendlo', 'date': '2023-12-06', 'time': '18:30:00', 'genre': 'Rock', 'venue': 'The Moroccan Lounge', 'city': 'Los Angeles', 'state': 'CA', 'long': '-118.23365', 'lat': '34.04898'}, {'Performing': "The Gang's All Here Tour With Skid Row And Buckcherry", 'date': '2023-12-06', 'time': '18:30:00', 'genre': 'Metal', 'venue': 'House of Blues Anaheim', 'city': 'Anaheim', 'state': 'CA', 'long': '-117.9229975', 'lat': '33.8095868'}, {'Performing': 'Your Neighbors with special guest 7KY', 'date': '2023-12-06', 'time': '19:00:00', 'genre': 'Rock', 'venue': 'The Peppermint Club', 'city': 'Los Angeles', 'state': 'CA', 'long': '-118.381072', 'lat': '34.076851'}, {'Performing': 'Amber Wild with Hardcastle, and Levi Evans', 'date': '2023-12-06', 'time': '19:00:00', 'genre': 'Rock', 'venue': 'The Echo', 'city': 'Los Angeles', 'state': 'CA', 'long': '-118.260108', 'lat': '34.077729'}, {'Performing': "Jenny Lewis - The Joy'All Ball Tour", 'date': '2023-12-06', 'time': 

In [67]:
#confirm both pages were extracted
len(la_list)

299

In [70]:
# Convert la_shows data into a Pandas DataFrame
la_data_df = pd.DataFrame(la_list)
# confirm Record Count
la_data_df.count()

Performing    299
date          299
time          299
genre         299
venue         299
city          299
state         299
long          299
lat           299
dtype: int64

In [71]:
# Display la_shows df
la_data_df.head()

,Performing,date,time,genre,venue,city,state,long,lat
0,Wendlo,2023-12-06,18:30:00,Rock,The Moroccan Lounge,Los Angeles,CA,-118.23365,34.04898
1,The Gang's All Here Tour With Skid Row And Buc...,2023-12-06,18:30:00,Metal,House of Blues Anaheim,Anaheim,CA,-117.9229975,33.8095868
2,Your Neighbors with special guest 7KY,2023-12-06,19:00:00,Rock,The Peppermint Club,Los Angeles,CA,-118.381072,34.076851
3,"Amber Wild with Hardcastle, and Levi Evans",2023-12-06,19:00:00,Rock,The Echo,Los Angeles,CA,-118.260108,34.077729
4,Jenny Lewis - The Joy'All Ball Tour,2023-12-06,19:00:00,Rock,The Observatory,Santa Ana,CA,-117.9175717,33.6997648


In [72]:
# Export the la_shows df into a csv
la_data_df.to_csv("../cities_data/la_show_data.csv", index_label="Show_ID")

In [76]:
nash_query = f"{url}apikey={ticketmaster_key}&classificationName=music&startDateTime={begin_dt}&endDateTime={end_dt}&size=200&dmaId=343&sort=date,asc"
nash_shows = requests.get(nash_query).json()
nash_list = []
#what we need...
for show in nash_shows['_embedded']['events']:
    nash_name = show["name"]
    #image = nyc_shows["image"[0]] 
    #^^can we save/retrieve image? it would be a nice touch...^^
    nash_show_date = show['dates']['start']['localDate'] #done
    try:
        #show['genre']
        nash_show_time = show['dates']['start']['localTime'] #done
    except KeyError:
        print('Undefined')
    #^^we need to see what we can do to stil include music performances even though no time is announced...^^
    nash_show_genre = show['classifications'][0]['genre']['name'] #done
    ##show_subgenre = show['classifications'][0]['subGenre']['name'] #done
    #^^how do we make an exception or skip if the subgenre is not applicable?^^
    nash_show_venue = show['_embedded']['venues'][0]['name'] #done
    nash_venue_city = show['_embedded']['venues'][0]['city']['name'] #done
    nash_venue_state = show['_embedded']['venues'][0]['state']['stateCode'] #done
    #^^in case we need these(city, state)..^^
    nash_venue_long = show['_embedded']['venues'][0]['location']['longitude'] #done
    nash_venue_lat = show['_embedded']['venues'][0]['location']['latitude'] #done
    nash_performances = {
        'Performing': nash_name,
        'date': nash_show_date,
        'time': nash_show_time,
        'genre': nash_show_genre,
        #'subgenre': show_subgenre,
        'venue': nash_show_venue,
        'city': nash_venue_city,
        'state': nash_venue_state,
        'long': nash_venue_long,
        'lat': nash_venue_lat,
    }
    nash_list.append(nash_performances)
print(nash_list)

[{'Performing': 'Dirty Dancing in Concert', 'date': '2023-12-06', 'time': '19:30:00', 'genre': 'Other', 'venue': 'Knoxville Civic Auditorium', 'city': 'Knoxville', 'state': 'TN', 'long': '-83.911513', 'lat': '35.965101'}, {'Performing': 'The Gatlin Brothers "Country and Christmas"', 'date': '2023-12-06', 'time': '19:30:00', 'genre': 'Country', 'venue': 'Bijou Theatre', 'city': 'Knoxville', 'state': 'TN', 'long': '-83.917015', 'lat': '35.962335'}, {'Performing': 'Trans-Siberian Orchestra-The Ghosts Of Christmas Eve', 'date': '2023-12-06', 'time': '19:00:00', 'genre': 'Rock', 'venue': 'Bridgestone Arena', 'city': 'Nashville', 'state': 'TN', 'long': '-86.77857', 'lat': '36.159949'}, {'Performing': 'Angel Olsen', 'date': '2023-12-06', 'time': '20:00:00', 'genre': 'Folk', 'venue': 'Brooklyn Bowl Nashville', 'city': 'Nashville', 'state': 'TN', 'long': '-86.783632', 'lat': '36.173214'}, {'Performing': 'Trans-Siberian Orchestra - The Ghosts Of Christmas Eve', 'date': '2023-12-07', 'time': '19:

In [77]:
# Convert nash_shows data into a Pandas DataFrame
nash_data_df = pd.DataFrame(nash_list)
# confirm Record Count
nash_data_df.count()

Performing    97
date          97
time          97
genre         97
venue         97
city          97
state         97
long          97
lat           97
dtype: int64

In [78]:
# Display nash_shows df
nash_data_df.head()

,Performing,date,time,genre,venue,city,state,long,lat
0,Dirty Dancing in Concert,2023-12-06,19:30:00,Other,Knoxville Civic Auditorium,Knoxville,TN,-83.911513,35.965101
1,"The Gatlin Brothers ""Country and Christmas""",2023-12-06,19:30:00,Country,Bijou Theatre,Knoxville,TN,-83.917015,35.962335
2,Trans-Siberian Orchestra-The Ghosts Of Christm...,2023-12-06,19:00:00,Rock,Bridgestone Arena,Nashville,TN,-86.77857,36.159949
3,Angel Olsen,2023-12-06,20:00:00,Folk,Brooklyn Bowl Nashville,Nashville,TN,-86.783632,36.173214
4,Trans-Siberian Orchestra - The Ghosts Of Chris...,2023-12-07,19:00:00,Rock,Food City Center,Knoxville,TN,-83.925366,35.9517627


In [79]:
# Export the nash_shows df into a csv
nash_data_df.to_csv("../cities_data/nash_show_data.csv", index_label="Show_ID")

In [80]:
aus_query = f"{url}apikey={ticketmaster_key}&classificationName=music&startDateTime={begin_dt}&endDateTime={end_dt}&size=200&dmaId=222&sort=date,asc"
aus_shows = requests.get(aus_query).json()
aus_list = []
#what we need...
for show in aus_shows['_embedded']['events']:
    aus_name = show["name"]
    #image = nyc_shows["image"[0]] 
    #^^can we save/retrieve image? it would be a nice touch...^^
    aus_show_date = show['dates']['start']['localDate'] #done
    try:
        #show['genre']
        aus_show_time = show['dates']['start']['localTime'] #done
    except KeyError:
        print('Undefined')

    aus_show_genre = show['classifications'][0]['genre']['name'] #done
    ##show_subgenre = show['classifications'][0]['subGenre']['name'] #done
    #^^how do we make an exception or skip if the subgenre is not applicable?^^
    aus_show_venue = show['_embedded']['venues'][0]['name'] #done
    aus_venue_city = show['_embedded']['venues'][0]['city']['name'] #done
    aus_venue_state = show['_embedded']['venues'][0]['state']['stateCode'] #done
    aus_venue_long = show['_embedded']['venues'][0]['location']['longitude'] #done
    aus_venue_lat = show['_embedded']['venues'][0]['location']['latitude'] #done
    aus_performances = {
        'Performing': aus_name,
        'date': aus_show_date,
        'time': aus_show_time,
        'genre': aus_show_genre,
        #'subgenre': show_subgenre,
        'venue': aus_show_venue,
        'city': aus_venue_city,
        'state': aus_venue_state,
        'long': aus_venue_long,
        'lat': aus_venue_lat,
    }
    aus_list.append(aus_performances)
print(aus_list)

[{'Performing': "Stayin' Alive: One Night of the Bee Gees", 'date': '2023-12-06', 'time': '19:30:00', 'genre': 'Undefined', 'venue': 'Paramount Theatre for the Performing Arts', 'city': 'Austin', 'state': 'TX', 'long': '-97.741848', 'lat': '30.26939'}, {'Performing': "JD McPherson- SOCKS: A Rock N' Roll Christmas Tour w/ Joel Paterson", 'date': '2023-12-06', 'time': '20:00:00', 'genre': 'Rock', 'venue': "Antone's Nightclub", 'city': 'Austin', 'state': 'TX', 'long': '-97.740401', 'lat': '30.26605'}, {'Performing': 'D.R.I.', 'date': '2023-12-07', 'time': '19:00:00', 'genre': 'Rock', 'venue': 'Come and Take It Live', 'city': 'Austin', 'state': 'TX', 'long': '-97.713303', 'lat': '30.228399'}, {'Performing': "JD McPherson- SOCKS: A Rock N' Roll Christmas Tour ", 'date': '2023-12-07', 'time': '20:00:00', 'genre': 'Rock', 'venue': "Antone's Nightclub", 'city': 'Austin', 'state': 'TX', 'long': '-97.740401', 'lat': '30.26605'}, {'Performing': 'Andy Frasco', 'date': '2023-12-07', 'time': '20:00:

In [81]:
# Convert aus_shows data into a Pandas DataFrame
aus_data_df = pd.DataFrame(aus_list)
# confirm Record Count
aus_data_df.count()

Performing    61
date          61
time          61
genre         61
venue         61
city          61
state         61
long          61
lat           61
dtype: int64

In [82]:
# Display aus_shows df
aus_data_df.head()

,Performing,date,time,genre,venue,city,state,long,lat
0,Stayin' Alive: One Night of the Bee Gees,2023-12-06,19:30:00,Undefined,Paramount Theatre for the Performing Arts,Austin,TX,-97.741848,30.26939
1,JD McPherson- SOCKS: A Rock N' Roll Christmas ...,2023-12-06,20:00:00,Rock,Antone's Nightclub,Austin,TX,-97.740401,30.26605
2,D.R.I.,2023-12-07,19:00:00,Rock,Come and Take It Live,Austin,TX,-97.713303,30.228399
3,JD McPherson- SOCKS: A Rock N' Roll Christmas ...,2023-12-07,20:00:00,Rock,Antone's Nightclub,Austin,TX,-97.740401,30.26605
4,Andy Frasco,2023-12-07,20:00:00,Alternative,The Mohawk-Austin,Austin,TX,-97.742401,30.2721


In [83]:
# Export the nash_shows df into a csv
aus_data_df.to_csv("../cities_data/aus_show_data.csv", index_label="Show_ID")

In [4]:
# Read saved data
# _________ ##= pd.read_csv("../output_data/dc_shows.csv", index_col="Show_ID")

# Display sample data
# ________ ##.head()